# Understanding the spread of COVID-19 based on European traffic data during summer

## 1. Introduction

### 1.1. Data

- Our World in Data COVID-19 dataset
    - Data on deaths in different countries.


- Google's traffic data 
    - Google provides anonymized insights from products such as Google Maps for researchers to help them to make critical analysis to combat COVID-19. 
    - Google has divided their traffic data into six traffic components: 
        1. retail \& recreation
            - places like restaurants, cafes, shopping centers, theme parks, museums, libraries and movie theaters
        2. grocery \& pharmacy
            - places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores and pharmacies
        3. parks
            - places like national parks, public beaches, marinas, dog parks, plazas and public gardens
        4. transit stations
            - places like public transport hubs such as subway, bus and train stations
        5. workplaces 
            - places of work
        6. residential 
            -  places of residence
     
  - These components do not tell anything how much time people spend in each section on average but they still give a lot of information how people's traffic behavior changed during the pandemic
 
 - The traffic data's baseline is counted as a median value of multiple days. Day-to-day changes should not be emphasized too much because they are effected on many different factors, f.e. the weather and public events.  




### 1.2. Moral behind the Bayesian model


- Getting the traffic data down slows down the spread of the virus. 
    - Several articles (Ferretti et al 2020, ECDC report, LSHTM report) have pointed out pre-symptomatic and asymptomatic  infections  play  a  significant  role  in  the  spread  of  COVID-19.  Indeed,  this observation  is  an  argument  it  may  not  be  enough  to  get  the  symptomatic  cases  to  stay at  home. Also  governmental  restrictions  should  be  implemented  to  get people’s movement down and furthermore the pandemic under control.
    - Essentially, the reason to implement non-pharmaceutical interventions is to get people's traffic data down!


- There are multiple reasons why it makes sense to analyse European countries in this research
    - COVID-19 hit European countries badly during autumn
    - European governments have similar capabilities to restrict their citizens movement in comparison to many countries, f.e. China
    - European countries adapted suppression strategy instead of mitigation one
    
    
- There are many major differences between European countries which effect on the spread of COVID-19
    - Examples: different age distributions, different population densities in cities, cultural differences
    - Therefore comparisons between European countries should be avoided

    
- The COVID-19-case data is not reliable at least as the only measure about the development of the epidemic. COVID-19-death data has many benefits compared to the COVID-19 case data! 
    - The amount of testing varies a lot between countries
    - Also using death data over infected data has the benefit that deaths measures much better country's success against the epidemic than infections
    





### 1.3. Motivation, research goals


1. To create a model which predicts the spread of the epidemic well based on the traffic data and data on deaths.

2. Based on the created model, trying to understand which of these Google's traffic components predicted the spread of COVID-19 in different European countries


### 1.4.  Structure



1. Introduction
    - Describes the essentials of this notebook
2. Getting an overview how COVID-19-cases and people's traffic behavior developed during the pandemic
    - This section gives moral for sections 4-8.
3. Dividing regions inside European countries in groups based on the development of the epidemic
    - COVID-19 hit European countries very differently. Therefore they are divided in three different groups.
4. Based on countries which did well against the epidemic during summer, understanding the impact of different traffic components to the spread of the epidemic
    - This section uses the assumption: The spread of COVID-19 can be predicted with people's traffic behavior.
    - This section gives understanding which Google traffic components had the biggest impact on the spread of COVID-19.
    - Important section, contains results!
5. Based on countries where the epidemic escalated during the summer, understanding the impact of different traffic components to the spread of the epidemic
    - This section has the same aim as the section 5 has. However, the methods used in these sections strongly differ from each other 
    - Important section, contains results!
6. Getting an overview if the traffic components with the most impact predicted well the spread of COVID-19 in different European countries
    - NOT YET IMPLEMENTED AT ALL
7. Summary
    - NOT YET IMPLEMENTED AT ALL
    - Summaries the whole notebook and also opens discussion about the findings

### 1.5. Libraries used in this notebook

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import linear_model

import datetime

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pystan
from scipy.integrate import odeint

import stan_utils

### 1.6. Parameters which need to be defined manually

- All the parameters, which need to be defined manually, are here


- During text there are detailed explanations what these parameters are

In [2]:
#############################################################################

# There is COVID-19-data from February until now
observations_start_date = datetime.datetime(2020, 2, 1, 0, 0)
observations_end_date = datetime.datetime(2020, 11, 15, 0, 0)

# However, the data analysis of this notebook concentrates on autumn months, i.e. on so called tail
tail_start_date = datetime.datetime(2020, 9, 1, 0, 0)


#############################################################################

# European countries ordered by population
european_countries = [
    'Germany', 'United Kingdom', 'France', 'Italy', 'Spain', 
    'Ukraine', 'Poland', 'Romania','Netherlands', 'Belgium', 
    'Greece', 'Sweden', 'Portugal', 'Hungary', 'Belarus', 
    'Austria', 'Switzerland', 'Bulgaria', 'Serbia', 'Denmark', 
    'Finland', 'Norway', 'Ireland', 'Croatia', 'Moldova', 
    'Bosnia and Herzegovina', 'Lithuania', 'Slovenia', 'Estonia' ]


#############################################################################

# Window size for convolution
w = 7

# Death limit
d = 1

In [3]:
# Follows directly from manual definitions 
num_countries = len(european_countries)
whole_interval_len = (observations_end_date - observations_start_date).days 
tail_interval_len = (observations_end_date - tail_start_date).days 
date_list = [observations_start_date + datetime.timedelta(days=x) for x in range(whole_interval_len)]

print("There are " + str(num_countries) + " European countries analysed in this notebook.")
print("The length of the whole interval: " + str(whole_interval_len))
print("The length of the tail interval: " + str(tail_interval_len))

There are 29 European countries analysed in this notebook.
The length of the whole interval: 288
The length of the tail interval: 75


#### Definition: Autumn interval $\tau_{\text{autumn}}$

Let's denote the autumn interval with $\tau_{\text{autumn}} = \{ 0,1,2, \dots, 75 \}$. Indeed $t = 0$ corresponds the date 1.9.2020, $t = 1$ corresponds the date 2.9.2020 and so on.

### 1.7. Adding previously cleaned data to dataframes 

#### Dataframe df_countries

- The length of this dataframe is 'num_countries'. Indeed, for each country there is one row.

In [4]:
# A dataframe sorted by countries

dtypes_countries = np.dtype([
          ('country', str),
          ('population', int),
          ('population_in_millions', int),
          ])

df_countries = pd.DataFrame(pd.read_csv('Cleaned_dataframes/df_countries.csv', dtype=dtypes_countries))

# Show the dataframe
df_countries

,country,population,population_in_millions
0,Germany,83783945,84
1,United Kingdom,67886004,68
2,France,65273512,65
3,Italy,60461828,60
4,Spain,46754783,47
5,Ukraine,43733759,44
6,Poland,37846605,38
7,Romania,19237682,19
8,Netherlands,17134873,17
9,Belgium,11589616,12


#### Dataframe df_days_by_countries

- All the days to which there exists traffic and infected data to each country


- The length of the dataframe equals 'num_countries' * 'whole_interval_len'



In [5]:
# A countrywise sorted dataframe s.t. for each day on the time interval of each country there is a row 

dtypes_days_by_countries = np.dtype([
          ('country', str),  # country name
          ('date', str), # current date. This will become datetime-time using parse_dates!
          ('new_deaths', int), # new deaths on that date
          ('new_deaths_smooth', int), # smoothened new deaths on that date
          ('total_deaths_per_million', float), # how many deaths per million has occured until that date
          ('traffic_retail', float), # retail and recreation traffic on that date
          ('traffic_supermarket', float), # supermarket and pharmacy traffic on that date
          ('traffic_parks', float),  # park traffic on that date
          ('traffic_transit_stations', float), # transit station traffic on that date
          ('traffic_workplaces', float), # workplace traffic on that date
          ('traffic_residential', float), # residential traffic on that date
          ])

df_days_by_countries = pd.DataFrame(pd.read_csv('Cleaned_dataframes/df_days_by_countries.csv', dtype=dtypes_days_by_countries))   

# Change the date-column from string-type to datetype
df_days_by_countries['date'] = pd.to_datetime(df_days_by_countries['date'], format='%Y-%m-%d')


#pd.set_option('display.max_rows', None)
# Show the dataframe
df_days_by_countries

,country,date,new_deaths,new_deaths_smooth,total_deaths_per_million,traffic_retail,traffic_supermarket,traffic_parks,traffic_transit_stations,traffic_workplaces,traffic_residential
0,Germany,2020-02-01,0,0,0.000,1.00,1.00,1.00,1.00,1.00,1.00
1,Germany,2020-02-02,0,0,0.000,1.00,1.00,1.00,1.00,1.00,1.00
2,Germany,2020-02-03,0,0,0.000,1.00,1.00,1.00,1.00,1.00,1.00
3,Germany,2020-02-04,0,0,0.000,1.00,1.00,1.00,1.00,1.00,1.00
4,Germany,2020-02-05,0,0,0.000,1.00,1.00,1.00,1.00,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...
8347,Estonia,2020-11-10,1,0,57.292,0.96,1.05,1.10,0.86,0.88,1.04
8348,Estonia,2020-11-11,0,0,57.292,0.96,1.06,1.06,0.85,0.88,1.04
8349,Estonia,2020-11-12,0,0,57.292,0.93,1.03,1.03,0.81,0.87,1.05
8350,Estonia,2020-11-13,0,0,57.292,0.90,1.04,0.93,0.78,0.86,1.05


## 2. Getting an overview how COVID-19-cases, -deaths and people's traffic behavior developed during the pandemic

### 2.1. Plot all Google's traffic components and also death and infected data countrywise

- The vertical black line represents when the tail of the pandemic starts 

In [32]:
# Define categories which are plotted
traffic_components = ['traffic_retail', 'traffic_supermarket', 'traffic_parks', 
                      'traffic_transit_stations', 'traffic_workplaces', 'traffic_residential'
                     ]

description = ['traffic in retail and recreation', 'traffic in supermarkets and pharmacy', 'traffic in parks',
               'traffic in transit stations', 'traffic in workplaces', 'traffic in residential ares'
              ]


# Loop over each country
for i in range(num_countries):
    
    # Define the current country, a temporary dataframe of the country and x-axis (dates)
    current_country = european_countries[i]
    df_current = df_days_by_countries[(df_days_by_countries['country'] == current_country)]
    x = df_current['date'].tolist() 
    
    print('\033[1m' + current_country)
    
    # Loop over each traffic component
    for j in range(len(traffic_components)):

        # Define y-components which are going to be plotted in one figure
        y_traffic = df_current[traffic_components[j]].tolist()
        y_deaths = df_current['new_deaths'].tolist()

        # Define the figure and different y-axis (there are 3 in total: traffic, infected, deaths)
        fig, host = plt.subplots(figsize=(26, 6))
        fig.subplots_adjust(right=0.75)
        par1 = host.twinx()

        # Set the most right one y-axis to right
        par1.spines["right"].set_position(("axes", 1.1))

        # Plot the traffic, infected and death data
        p1, = host.plot(x, y_traffic, "b-", label='The change in ' + description[j] +  ' (%)' )
        p2, = par1.plot(x, y_deaths, marker = 'o', linestyle='', color = "black", label="New deaths ()")

        # Define the texts
        host.set_ylabel('The change in ' + description[j] +  ' (%)')
        par1.set_ylabel("New deaths ()")

        # Text on the axis with the correct color
        host.yaxis.label.set_color(p1.get_color())
        par1.yaxis.label.set_color(p2.get_color())

        # Make little spikes for different y-axis
        tkw = dict(size=30, width=1.6)
        host.tick_params(axis='y', colors=p1.get_color(), **tkw)
        par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
        host.tick_params(axis='x', **tkw)

        plt.axvline(tail_start_date, color='black')
        
        print('\033[0m' + description[j])
        plt.show()

### Conclusions

- Based on previous plots, clearly residential traffic does not impact negatively on the spread of COVID-19. The effect of other traffic components will be analysed more detailed and residential traffic data is not analysed any more in this project.
 

- It is difficult to analyse the impact of different traffic components to the spread of COVID-19 based on the first local maximum of the epidemic (in most countries it happend on March or on April)
    
    - Almost in every European country all the traffic data components except residential traffic went strongly down at the same time. Therefore, it is difficult to say which traffic component truely mattered based on the beginning of the pandemic.
    
    
- Therefore, let's concentrate the analysis what happend later during the second wave, i.e. concentrate on the tail of the pandemic, autumn which is defined to start on 1.9.2020!

## Bayesian model

### Theory behind the model

Let's start using SEIR-model. 

In this extended SEIR-model, the population $N \in \mathbb{N}$ is divided in four groups: susceptibles $S(t) \in \mathbb{N}$, exposed $E(t) \in \mathbb{N}$, infected $I(t) \in \mathbb{N}$ and removed $R(t) \in \mathbb{N}$. Notice that the group removed $R$ includes everyone who is either recovered from the disease, isolated or died because of it. The changes of these groups can be described by the following equations:

\begin{equation}
\frac{\partial S}{\partial t} = - \frac{\beta S I }{N}
\end{equation}

\begin{equation}
\frac{\partial E}{\partial t} =  \frac{\beta S I}{N} - \frac{E}{D_e}
\end{equation}

\begin{equation}
\frac{\partial I}{\partial t} =  \frac{E}{D_e} - \frac{I}{D_i}
\end{equation}

\begin{equation}
\frac{\partial R}{\partial t} =  \frac{I}{D_i}
\end{equation}

$D_e \in \mathbb{R}$ is the latent period of the disease. 
Furthermore, $D_i \in \mathbb{R}$ is the period how long both symptomatic and asymptomatic cases are going to spread the disease before getting noninfectious, dead or isolated from other population. 

Furthermore, $\beta(t)$ is the average daily rate how many people the infected $I$ are going to meet inside the population. We fix this rate with Google's traffic components with the equation 
$\beta(t) = \sum_{i \in \{1,2,3,4,5\}} c_i \tau_{t,i}$.
From this equation, the non-fixed parameters $c_1, c_2, c_3, c_4, c_5$ are estimated.


The daily death data is assumed to be Poisson distributed. 
Therefore, the parameters $c_1, c_2, c_3, c_4, c_5$ are estimated with the likelihood function 

\begin{equation}
L(c_1, c_2, c_3, c_4, c_5) = \prod_{i=1}^{t} \frac{e^{- \lambda_i} \lambda_i^{x_i}}{x_i !}
\end{equation}

In the previous equation $x_i$ is the smoothened deaths at day $i$. Also, $\lambda_i = \frac{\alpha I}{D_i}$ is the rate of the Poisson distribution where $\alpha \in [0,1]$ is the proportion of infected who die. It is good to notice $I(t)$ depends on values of $c_1, c_2, c_3, c_4, c_5$. 

Let's use non-informative priors and MCMC-methods.

### Practical implementation using STAN

In [6]:
t = []
for i in range(tail_interval_len):
    if ((i + 1) % 7 == 0):
        t.append(i+1)
t

[7, 14, 21, 28, 35, 42, 49, 56, 63, 70]

In [7]:
deaths_germany = df_days_by_countries[(df_days_by_countries['country'] == 'Germany') &
                  (df_days_by_countries['date'] >= tail_start_date)]['new_deaths_smooth'].tolist()

deaths_germany = [deaths_germany[i] for i in t] 

print(len(deaths_germany))
print(deaths_germany)

10
[3, 4, 4, 10, 10, 12, 28, 37, 80, 120]


In [8]:
retail_data_germany = df_days_by_countries[(df_days_by_countries['country'] == 'Germany') &
                  (df_days_by_countries['date'] >= tail_start_date)]['traffic_retail'].tolist()
retail_data_germany = [retail_data_germany[i] for i in t] 
print(len(retail_data_germany))
print(retail_data_germany)

transport_data_germany = df_days_by_countries[(df_days_by_countries['country'] == 'Germany') &
                  (df_days_by_countries['date'] >= tail_start_date)]['traffic_transit_stations'].tolist()
transport_data_germany = [transport_data_germany[i] for i in t] 
print(len(transport_data_germany))
print(transport_data_germany)

10
[1.0, 0.99, 0.96, 0.92, 0.93, 0.94, 0.89, 0.86, 0.73, 0.73]
10
[0.88, 0.9, 0.89, 0.76, 0.86, 0.8, 0.74, 0.8, 0.74, 0.74]


In [13]:
sm = stan_utils.StanModel_cache("./seir_model.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1fd928b1c3f5d28e7940494b27230ea4 NOW.


In [15]:
n_days = len(deaths_germany)
N = df_countries[(df_countries['country'] == 'Germany')]['population'][0]

e0 = 100
i0 = 100
r0 = 1000

y0 = [N - e0 - i0 - r0, e0, i0, r0]
t0 = 0
t = t

D_e = 5.3
D_i = 5
alpha = 0.01

traffic1 = retail_data_germany
traffic2 = transport_data_germany
deaths = deaths_germany

seir_data = {
    "n_days": n_days,
    "y0": y0,
    "t0": t0,
    "ts": t,
    "N": N,
    "D_e": D_e,
    "D_i": D_i,
    "alpha": alpha,
    "traffic1": traffic1,
    "traffic2": traffic2,
    "deaths": deaths
}

fit = sm.sampling(seir_data)
samples = fit.extract(permuted=True)

In [16]:
fit

Inference for Stan model: anon_model_1fd928b1c3f5d28e7940494b27230ea4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
c[1]         5.33    0.05   0.99   3.35   4.66   5.33   6.02   7.22    413    1.0
c[2]        -4.76    0.05    1.0  -6.66  -5.45  -4.77   -4.1  -2.78    413    1.0
y[1,1]      8.4e7   13.82 283.06  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    419    1.0
y[2,1]      8.4e7   43.32 888.65  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    421    1.0
y[3,1]      8.4e7   75.46 1551.8  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    423    1.0
y[4,1]      8.4e7   117.7 2432.9  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    427    1.0
y[5,1]      8.4e7   169.7 3548.4  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    437    1.0
y[6,1]      8.4e7  227.51 4886.5  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    461    1.0
y[7,1]      8.4e7  277.54 6399.0  8.4e7  8.4e7  8.4e7  8.4e7  8.4e7    53

##  Summary

###  The findings of this research

### An analysis of potential mistakes

- Different traffic components correlate with another which adds noise
    - Example: A person uses public transport, goes to the supermarket, goes to a park and travels back home

### Discussion 

- The results which traffic components have the biggest impact on the spread of COVID-19 are intuitive: 
    - In places like supermarkets and pharmacies people can hold distance with other people relatively well.
    - On contrary, in public transports people easily go very close to other people.
    - etc....

### 7.4. Good articles

- https://www.medrxiv.org/content/10.1101/2020.03.03.20030593v1.supplementary-material
    - This has a great supplementary material section, and our model is based on that

- https://www.fhi.no/en/id/infectious-diseases/coronavirus/coronavirus-modelling-at-the-niph-fhi/
    - The Norwegian Institute of Public Health has a great report of modeling COVID-19 with extended SEIR-model 

- https://www.medrxiv.org/content/10.1101/2020.05.28.20116129v3.full.pdf
    - More like nice to read to get some ideas, not direct connection to our model